In [ ]:
# ===============================================================================
# CELL 1: Dependencies & WandB Setup
# ===============================================================================
print("Installing dependencies...")
!pip install -q torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu121
!pip install -q wandb scikit-learn matplotlib seaborn tqdm Pillow rasterio pandas

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torch.cuda.amp import autocast, GradScaler
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import rasterio
import wandb
import os
import gc
import time
import warnings
warnings.filterwarnings('ignore')

# WandB Login
wandb.login(key="5424a3d65aac1662f5be82d4439aaac35046689e")
print("✓ WandB authenticated")

# Device Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_count = torch.cuda.device_count()
print(f"\n🚀 Device: {device} | GPUs: {gpu_count}")
if gpu_count > 0:
    for i in range(gpu_count):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")

# MAXIMIZED Config for full 12h utilization with extensive evaluation
config = {
    # Dataset - MAXIMIZED
    'dataset_size': 100000,  # Increased from 60k → 100k (use full dataset)
    'num_classes': 19,       # BigEarthNet-19 classes
    
    # SR Model
    'sr_model_path': '/kaggle/input/sr-model/pytorch/default/3/generator_ensemble.pth',
    'lr_size': 32,           # LR input size
    'hr_size': 128,          # HR output size (32*4)
    
    # Classifier Training - MAXIMIZED FOR 12H
    'clf_epochs': 50,        # Increased from 30 → 50 for maximum learning
    'batch_size': 64,        # Increased from 48 → 64 (optimal throughput)
    'lr': 3e-4,              # Increased from 2e-4 → 3e-4 (faster convergence)
    'weight_decay': 1e-5,    # Reduced from 5e-5 → 1e-5 (minimal regularization)
    'warmup_epochs': 5,      # Increased from 3 → 5 (better stability)
    'label_smoothing': 0.15, # Increased from 0.1 → 0.15 (more smoothing)
    
    # Active Learning
    'al_cycles': 4,
    'al_epochs_per_cycle': 10,
    'initial_labeled_ratio': 0.1,
    'query_size_ratio': 0.1,
    
    # Training Enhancements
    'num_workers': 4,
    'pin_memory': True,
    'mixed_precision': True,
    'gradient_clip': 1.0,
    'ema_decay': 0.9995,     # Increased decay for more stability
    
    # Evaluation & Visualization
    'eval_frequency': 1,     # Evaluate every epoch
    'viz_frequency': 2,      # Generate visualizations every 2 epochs
    'save_feature_maps': True,  # Save feature extraction visualizations
    'save_sr_examples': True,   # Save SR enhancement examples
}

# Initialize WandB
wandb.init(
    project="SR-ResNet-AL-Classification",
    config=config
    # name parameter removed - WandB will generate random name automatically
)

print("\n✓ Setup complete!")
print(f"Config: {config}")

Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 481.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 20.8 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.1.0+cu121 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 75.5 MB/s eta 0:00:00:00:0100:01


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hegdesudarshan (hegdesudarshan-hegde) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✓ WandB authenticated

🚀 Device: cuda | GPUs: 2
   GPU 0: Tesla T4
   GPU 1: Tesla T4



✓ Setup complete!
Config: {'dataset_size': 50000, 'num_classes': 19, 'sr_model_path': '/kaggle/input/sr-model/pytorch/default/3/generator_ensemble.pth', 'lr_size': 32, 'hr_size': 128, 'clf_epochs': 20, 'batch_size': 32, 'lr': 0.0001, 'weight_decay': 0.0001, 'al_cycles': 4, 'al_epochs_per_cycle': 10, 'initial_labeled_ratio': 0.1, 'query_size_ratio': 0.1, 'num_workers': 4, 'pin_memory': True, 'mixed_precision': True}


In [2]:
# ===============================================================================
# CELL 2: Load Pre-trained SR Model (EXACT ARCHITECTURE FROM CHECKPOINT)
# ===============================================================================

class RFB(nn.Module):
    """Receptive Field Block - EXACT match to checkpoint"""
    def __init__(self, in_channels=64):
        super().__init__()
        # Branch 1: AvgPool(3) + Conv + ReLU + Conv
        self.branch1 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_channels, 16, 1, 1, 0),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, 3, 1, padding=1, dilation=1),
            nn.ReLU(inplace=True)
        )
        # Branch 2: AvgPool(5) + Conv + ReLU + Conv
        self.branch2 = nn.Sequential(
            nn.AvgPool2d(5, stride=1, padding=2),
            nn.Conv2d(in_channels, 24, 1, 1, 0),
            nn.ReLU(inplace=True),
            nn.Conv2d(24, 24, 3, 1, padding=2, dilation=2),
            nn.ReLU(inplace=True)
        )
        # Branch 3: AvgPool(7) + Conv + ReLU + Conv
        self.branch3 = nn.Sequential(
            nn.AvgPool2d(7, stride=1, padding=3),
            nn.Conv2d(in_channels, 24, 1, 1, 0),
            nn.ReLU(inplace=True),
            nn.Conv2d(24, 24, 3, 1, padding=3, dilation=3),
            nn.ReLU(inplace=True)
        )
        # Changed to match checkpoint: conv_concat instead of conv
        self.conv_concat = nn.Sequential(
            nn.Conv2d(64, 64, 1, 1, 0)
        )
        
    def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        out = torch.cat([b1, b2, b3], 1)
        return self.conv_concat(out) * 0.2 + x

class DenseBlock(nn.Module):
    """Dense Block with 5 conv layers - MODIFIED channel counts to match checkpoint"""
    def __init__(self, nf=64):
        super().__init__()
        # Changed: nf=64 in Generator, but DenseBlock uses nf_internal=32
        nf_internal = 32
        self.conv1 = nn.Conv2d(nf, nf_internal, 3, 1, 1)
        self.conv2 = nn.Conv2d(nf + nf_internal, nf_internal, 3, 1, 1)
        self.conv3 = nn.Conv2d(nf + nf_internal*2, nf_internal, 3, 1, 1)
        self.conv4 = nn.Conv2d(nf + nf_internal*3, nf_internal, 3, 1, 1)
        self.conv5 = nn.Conv2d(nf + nf_internal*4, nf, 3, 1, 1)
        
    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(torch.cat([x, x1], 1)))
        x3 = F.relu(self.conv3(torch.cat([x, x1, x2], 1)))
        x4 = F.relu(self.conv4(torch.cat([x, x1, x2, x3], 1)))
        x5 = self.conv5(torch.cat([x, x1, x2, x3, x4], 1))
        return x5 * 0.2 + x

class RRDB(nn.Module):
    """Residual-in-Residual Dense Block (3 DenseBlocks)"""
    def __init__(self, nf=64):
        super().__init__()
        self.db1 = DenseBlock(nf)
        self.db2 = DenseBlock(nf)
        self.db3 = DenseBlock(nf)
        
    def forward(self, x):
        out = self.db3(self.db2(self.db1(x)))
        return out * 0.2 + x

class RRFDB(nn.Module):
    """Residual RFB Dense Block - MODIFIED to match checkpoint structure"""
    def __init__(self, nf=64):
        super().__init__()
        # Changed: Use named attributes instead of ModuleList to match checkpoint keys
        self.rfb1 = RFB(nf)
        self.rfb2 = RFB(nf)
        self.rfb3 = RFB(nf)
        self.rfb4 = RFB(nf)
        self.rfb5 = RFB(nf)
        
    def forward(self, x):
        out = self.rfb1(x)
        out = self.rfb2(out)
        out = self.rfb3(out)
        out = self.rfb4(out)
        out = self.rfb5(out)
        return out * 0.2 + x

class Generator(nn.Module):
    """Generator: 12 RRDB + 6 RRFDB + 8x upscale - EXACT architecture from checkpoint"""
    def __init__(self, num_rrdb=12, num_rrfdb=6, nf=64):
        super().__init__()
        self.conv_first = nn.Conv2d(3, nf, 3, 1, 1)
        
        # Trunk A: 12 RRDB blocks
        self.trunk_a = nn.Sequential(*[RRDB(nf) for _ in range(num_rrdb)])
        
        # Trunk RFB: 6 RRFDB blocks
        self.trunk_rfb = nn.Sequential(*[RRFDB(nf) for _ in range(num_rrfdb)])
        
        # RFB upsampling
        self.rfb_up = RFB(nf)
        
        # 8x upscaling (3 PixelShuffle layers: 2x each = 2^3 = 8x)
        self.upsample = nn.Sequential(
            nn.Conv2d(nf, nf*4, 3, 1, 1),
            nn.PixelShuffle(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(nf, nf*4, 3, 1, 1),
            nn.PixelShuffle(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(nf, nf*4, 3, 1, 1),
            nn.PixelShuffle(2),
            nn.ReLU(inplace=True)
        )
        
        # Changed to match checkpoint: Sequential with conv_final layers
        self.conv_final = nn.Sequential(
            nn.Conv2d(nf, nf, 3, 1, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(nf, 3, 3, 1, 1)
        )
        
    def forward(self, x):
        feat = self.conv_first(x)
        trunk_a_out = self.trunk_a(feat)
        trunk_rfb_out = self.trunk_rfb(trunk_a_out)
        rfb_up_out = self.rfb_up(trunk_rfb_out)
        up = self.upsample(rfb_up_out + feat)
        return torch.tanh(self.conv_final(up))

# Load Pre-trained SR Model
print("Loading pre-trained SR model...")
sr_model = Generator(num_rrdb=12, num_rrfdb=6, nf=64).to(device)

try:
    state_dict = torch.load(config['sr_model_path'], map_location=device)
    sr_model.load_state_dict(state_dict)
    sr_model.eval()
    
    # Test SR model
    with torch.no_grad():
        test_input = torch.randn(1, 3, 32, 32).to(device)
        test_output = sr_model(test_input)
        print(f"✓ SR Model loaded: {test_input.shape} → {test_output.shape}")
        
    # Count parameters
    total_params = sum(p.numel() for p in sr_model.parameters())
    print(f"  Parameters: {total_params/1e6:.2f}M")
    
except Exception as e:
    print(f"❌ Error loading SR model: {e}")
    import traceback
    traceback.print_exc()
    raise

# Freeze SR model (no training needed)
for param in sr_model.parameters():
    param.requires_grad = False

print("\n✓ SR model ready for inference!")


Loading pre-trained SR model...
✓ SR Model loaded: torch.Size([1, 3, 32, 32]) → torch.Size([1, 3, 256, 256])
  Parameters: 9.77M

✓ SR model ready for inference!


In [3]:
# ===============================================================================
# CELL 3: Dataset Loading (BigEarthNet)
# ===============================================================================

class BigEarthNetDataset(Dataset):
    """BigEarthNet dataset with SR preprocessing"""
    def __init__(self, root_path, patch_ids, patch_to_bands, patch_to_label, 
                 sr_model=None, phase='train'):
        self.root_path = root_path
        self.patch_ids = patch_ids
        self.patch_to_bands = patch_to_bands
        self.patch_to_label = patch_to_label
        self.sr_model = sr_model
        self.phase = phase
        
        # Transforms
        if phase == 'train':
            self.spatial_aug = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(90)
            ])
        else:
            self.spatial_aug = None
            
        self.to_tensor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
    
    def __len__(self):
        return len(self.patch_ids)
    
    def __getitem__(self, idx):
        patch_id = self.patch_ids[idx]
        bands = self.patch_to_bands[patch_id]
        
        try:
            # Load RGB bands (B04=Red, B03=Green, B02=Blue)
            b02 = rasterio.open(bands['02']).read(1).astype(np.float32) / 10000.0
            b03 = rasterio.open(bands['03']).read(1).astype(np.float32) / 10000.0
            b04 = rasterio.open(bands['04']).read(1).astype(np.float32) / 10000.0
            
            # Stack to RGB (120x120)
            hr_np = np.stack([b04, b03, b02], axis=-1)
            hr_np = np.clip(hr_np, 0, 1)
            
            # Convert to PIL for augmentation
            hr_pil = Image.fromarray((hr_np * 255).astype(np.uint8))
            
            # Apply spatial augmentation
            if self.spatial_aug:
                hr_pil = self.spatial_aug(hr_pil)
            
            # Resize to 32x32 for LR input
            lr_pil = hr_pil.resize((32, 32), Image.BICUBIC)
            
            # To tensor
            lr_tensor = self.to_tensor(lr_pil)
            
            # Get label (multi-hot → single label via argmax)
            label_multihot = self.patch_to_label.get(patch_id, torch.zeros(config['num_classes']))
            label = torch.argmax(label_multihot).long()
            
            return {
                'lr': lr_tensor,
                'label': label,
                'patch_id': patch_id
            }
            
        except Exception as e:
            # Return black dummy on error
            print(f"Error loading {patch_id}: {e}")
            return {
                'lr': torch.zeros(3, 32, 32),
                'label': torch.tensor(0, dtype=torch.long),
                'patch_id': patch_id
            }

# Load BigEarthNet metadata
print("Loading BigEarthNet dataset...")
image_root_path = '/kaggle/input/bigearthnetv2-s2-4/'

# Find all TIF files
import glob
all_tif_paths = glob.glob(os.path.join(image_root_path, '**/*.tif'), recursive=True)
print(f"Found {len(all_tif_paths)} band files")

# Group by patch ID
patch_to_bands = {}
for path in all_tif_paths:
    fname = os.path.basename(path)
    if '_B' in fname:
        patch_id = '_'.join(fname.split('_B')[:-1])
        band = fname.split('_B')[-1].split('.')[0]
        if patch_id not in patch_to_bands:
            patch_to_bands[patch_id] = {}
        patch_to_bands[patch_id][band] = path

# Filter patches with RGB bands
valid_patches = [pid for pid, bands in patch_to_bands.items() 
                 if all(b in bands for b in ['02', '03', '04'])]
valid_patches = valid_patches[:config['dataset_size']]
print(f"Valid RGB patches: {len(valid_patches)}")

# Load labels from metadata
metadata_path = os.path.join(image_root_path, 'metadata.parquet')
if os.path.exists(metadata_path):
    df = pd.read_parquet(metadata_path)
    patch_to_label = {}
    for _, row in df.iterrows():
        pid = row['patch_id']
        labels_list = row['labels'] if isinstance(row['labels'], list) else []
        multi_hot = torch.zeros(config['num_classes'])
        for lbl in labels_list:
            if 0 <= lbl < config['num_classes']:
                multi_hot[lbl] = 1.0
        if pid in valid_patches:
            patch_to_label[pid] = multi_hot
    print(f"Loaded labels for {len(patch_to_label)} patches")
else:
    print("Warning: metadata.parquet not found, using dummy labels")
    patch_to_label = {pid: torch.zeros(config['num_classes']) for pid in valid_patches}

# Train/Val split
train_ids, val_ids = train_test_split(valid_patches, test_size=0.2, random_state=42)
print(f"Split: {len(train_ids)} train, {len(val_ids)} val")

# Create datasets
train_dataset = BigEarthNetDataset(image_root_path, train_ids, patch_to_bands, 
                                   patch_to_label, sr_model, phase='train')
val_dataset = BigEarthNetDataset(image_root_path, val_ids, patch_to_bands, 
                                 patch_to_label, sr_model, phase='val')

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], 
                         shuffle=True, num_workers=config['num_workers'], 
                         pin_memory=config['pin_memory'])
val_loader = DataLoader(val_dataset, batch_size=config['batch_size']*2, 
                       shuffle=False, num_workers=config['num_workers'], 
                       pin_memory=config['pin_memory'])

print("\n✓ Dataset loaded!")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")

# Test batch
sample_batch = next(iter(train_loader))
print(f"\nSample batch shapes:")
print(f"  LR: {sample_batch['lr'].shape}")
print(f"  Label: {sample_batch['label'].shape}")

Loading BigEarthNet dataset...
Found 347244 band files
Valid RGB patches: 28937
Split: 23149 train, 5788 val

✓ Dataset loaded!
  Train batches: 724
  Val batches: 91

Sample batch shapes:
  LR: torch.Size([32, 3, 32, 32])
  Label: torch.Size([32])


In [4]:
# ===============================================================================
# CELL 3.5: GPU Memory Cleanup (Run this if you encounter CUDA errors)
# ===============================================================================

print("Cleaning up GPU memory...")
torch.cuda.empty_cache()
gc.collect()

# Reset CUDA device if needed
if torch.cuda.is_available():
    torch.cuda.synchronize()
    print(f"✓ GPU memory cleaned")
    for i in range(torch.cuda.device_count()):
        mem_allocated = torch.cuda.memory_allocated(i) / 1024**3
        mem_reserved = torch.cuda.memory_reserved(i) / 1024**3
        print(f"  GPU {i}: {mem_allocated:.2f}GB allocated, {mem_reserved:.2f}GB reserved")


Cleaning up GPU memory...
✓ GPU memory cleaned
  GPU 0: 0.07GB allocated, 0.08GB reserved
  GPU 1: 0.00GB allocated, 0.00GB reserved


In [ ]:
# ===============================================================================
# CELL 4: ResNet Classifier Definition
# ===============================================================================

class SREnhancedClassifier(nn.Module):
    """ResNet50-based classifier that processes SR-enhanced images"""
    def __init__(self, num_classes, sr_model, pretrained=True):
        super().__init__()
        self.sr_model = sr_model  # Frozen SR model
        
        # Load pretrained ResNet50
        if pretrained:
            weights = ResNet50_Weights.IMAGENET1K_V2
            self.backbone = resnet50(weights=weights)
        else:
            self.backbone = resnet50(weights=None)
        
        # Replace final FC layer with enhanced classifier head
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.4),  # Increased dropout
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, lr_images):
        # Step 1: SR enhancement (frozen) - 32x32 → 256x256
        with torch.no_grad():
            sr_images = self.sr_model(lr_images)
            # Resize 256x256 → 224x224 for ResNet50
            sr_images = F.interpolate(sr_images, size=(224, 224), mode='bilinear', align_corners=False)
        
        # Step 2: ResNet classification
        x = self.backbone.conv1(sr_images)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        
        x = self.backbone.avgpool(x)
        features = torch.flatten(x, 1)
        output = self.backbone.fc(features)
        return output
    
    def get_features(self, lr_images):
        """Extract features for active learning"""
        with torch.no_grad():
            sr_images = self.sr_model(lr_images)
            sr_images = F.interpolate(sr_images, size=(224, 224), mode='bilinear', align_corners=False)
            
            x = self.backbone.conv1(sr_images)
            x = self.backbone.bn1(x)
            x = self.backbone.relu(x)
            x = self.backbone.maxpool(x)
            
            x = self.backbone.layer1(x)
            x = self.backbone.layer2(x)
            x = self.backbone.layer3(x)
            x = self.backbone.layer4(x)
            
            x = self.backbone.avgpool(x)
            return torch.flatten(x, 1)

# Create classifier with error handling
print("Creating SR-Enhanced ResNet50 Classifier...")

try:
    # Clean GPU memory before creating model
    torch.cuda.empty_cache()
    gc.collect()
    
    # Create classifier on CPU first
    classifier = SREnhancedClassifier(config['num_classes'], sr_model, pretrained=True)
    
    # Move to GPU carefully
    classifier = classifier.to(device)
    print("✓ Classifier moved to GPU")
    
except RuntimeError as e:
    if "CUDA" in str(e):
        print(f"⚠ CUDA Error: {e}")
        print("Attempting recovery: Restarting kernel may help")
        print("Run the memory cleanup cell (Cell 3.5) and try again")
        raise
    else:
        raise

if gpu_count > 1:
    classifier = nn.DataParallel(classifier)
    print(f"  Using DataParallel across {gpu_count} GPUs")

# Count parameters
trainable_params = sum(p.numel() for p in classifier.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in classifier.parameters())
print(f"\n✓ Classifier created")
print(f"  Total parameters: {total_params/1e6:.2f}M")
print(f"  Trainable parameters: {trainable_params/1e6:.2f}M")

# Test forward pass
try:
    with torch.no_grad():
        test_lr = torch.randn(2, 3, 32, 32).to(device)
        test_output = classifier(test_lr)
        print(f"\nTest forward pass: {test_lr.shape} → {test_output.shape}")
        print(f"Output range: [{test_output.min():.3f}, {test_output.max():.3f}]")
except RuntimeError as e:
    print(f"⚠ Test forward pass failed: {e}")
    print("This may indicate GPU memory issues. Try restarting the kernel.")


Creating SR-Enhanced ResNet50 Classifier...


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s] 


✓ Classifier moved to GPU
  Using DataParallel across 2 GPUs

✓ Classifier created
  Total parameters: 33.32M
  Trainable parameters: 23.55M

Test forward pass: torch.Size([2, 3, 32, 32]) → torch.Size([2, 19])
Output range: [-0.144, 0.135]


In [6]:
# ===============================================================================
# CELL 5: Training & Evaluation Functions
# ===============================================================================

def train_epoch(model, loader, criterion, optimizer, scaler, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc="Training")
    for batch in pbar:
        lr_imgs = batch['lr'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        
        if config['mixed_precision']:
            with autocast():
                outputs = model(lr_imgs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(lr_imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item() * lr_imgs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'acc': f"{100.*correct/total:.2f}%"
        })
    
    return total_loss / total, 100. * correct / total

def evaluate(model, loader, criterion, device):
    """Evaluate model"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            lr_imgs = batch['lr'].to(device)
            labels = batch['label'].to(device)
            
            if config['mixed_precision']:
                with autocast():
                    outputs = model(lr_imgs)
                    loss = criterion(outputs, labels)
            else:
                outputs = model(lr_imgs)
                loss = criterion(outputs, labels)
            
            total_loss += loss.item() * lr_imgs.size(0)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    accuracy = accuracy_score(all_labels, all_preds) * 100
    f1 = f1_score(all_labels, all_preds, average='macro') * 100
    avg_loss = total_loss / len(all_labels)
    
    return avg_loss, accuracy, f1, all_preds, all_labels

def plot_confusion_matrix(y_true, y_pred, epoch):
    """Plot confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=False, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - Epoch {epoch}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    
    # Log to wandb
    wandb.log({"confusion_matrix": wandb.Image(plt)})
    plt.close()

print("✓ Training functions defined")

✓ Training functions defined


In [ ]:
# ===============================================================================
# CELL 5.5: Comprehensive Evaluation Metrics & Visualization Functions
# ===============================================================================

def compute_per_class_metrics(y_true, y_pred, num_classes=19):
    """Compute precision, recall, F1 per class"""
    from sklearn.metrics import precision_recall_fscore_support
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=list(range(num_classes)), zero_division=0
    )
    return precision, recall, f1, support

def plot_training_curves(history):
    """Plot training and validation curves"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history['train_loss'], label='Train', linewidth=2)
    axes[0, 0].plot(history['val_loss'], label='Val', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Loss Curves')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[0, 1].plot(history['train_acc'], label='Train', linewidth=2)
    axes[0, 1].plot(history['val_acc'], label='Val', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].set_title('Accuracy Curves')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # F1 Score
    axes[1, 0].plot(history['val_f1'], label='Val F1 (Macro)', linewidth=2, color='green')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('F1 Score (%)')
    axes[1, 0].set_title('F1 Score Curve')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Learning Rate
    axes[1, 1].plot(history['lr'], linewidth=2, color='red')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_title('Learning Rate Schedule')
    axes[1, 1].set_yscale('log')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    wandb.log({"training_curves": wandb.Image(plt)})
    plt.savefig('/kaggle/working/training_curves.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_per_class_performance(precision, recall, f1, support, class_names=None):
    """Plot per-class metrics"""
    if class_names is None:
        class_names = [f'Class {i}' for i in range(len(precision))]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    x = np.arange(len(class_names))
    
    # Precision
    axes[0, 0].bar(x, precision * 100, color='skyblue', edgecolor='navy', alpha=0.7)
    axes[0, 0].set_xlabel('Class')
    axes[0, 0].set_ylabel('Precision (%)')
    axes[0, 0].set_title('Per-Class Precision')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(class_names, rotation=45, ha='right', fontsize=8)
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    axes[0, 0].axhline(y=precision.mean()*100, color='red', linestyle='--', linewidth=2, label='Mean')
    axes[0, 0].legend()
    
    # Recall
    axes[0, 1].bar(x, recall * 100, color='lightgreen', edgecolor='darkgreen', alpha=0.7)
    axes[0, 1].set_xlabel('Class')
    axes[0, 1].set_ylabel('Recall (%)')
    axes[0, 1].set_title('Per-Class Recall')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(class_names, rotation=45, ha='right', fontsize=8)
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    axes[0, 1].axhline(y=recall.mean()*100, color='red', linestyle='--', linewidth=2, label='Mean')
    axes[0, 1].legend()
    
    # F1 Score
    axes[1, 0].bar(x, f1 * 100, color='lightcoral', edgecolor='darkred', alpha=0.7)
    axes[1, 0].set_xlabel('Class')
    axes[1, 0].set_ylabel('F1 Score (%)')
    axes[1, 0].set_title('Per-Class F1 Score')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(class_names, rotation=45, ha='right', fontsize=8)
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    axes[1, 0].axhline(y=f1.mean()*100, color='red', linestyle='--', linewidth=2, label='Mean')
    axes[1, 0].legend()
    
    # Support (sample count)
    axes[1, 1].bar(x, support, color='plum', edgecolor='purple', alpha=0.7)
    axes[1, 1].set_xlabel('Class')
    axes[1, 1].set_ylabel('Sample Count')
    axes[1, 1].set_title('Per-Class Sample Distribution')
    axes[1, 1].set_xticks(x)
    axes[1, 1].set_xticklabels(class_names, rotation=45, ha='right', fontsize=8)
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    wandb.log({"per_class_metrics": wandb.Image(plt)})
    plt.savefig('/kaggle/working/per_class_metrics.png', dpi=300, bbox_inches='tight')
    plt.close()

class EMA:
    """Exponential Moving Average for model weights"""
    def __init__(self, model, decay=0.999):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}
        self.register()
    
    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()
    
    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()
    
    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.backup[name] = param.data.clone()
                param.data = self.shadow[name]
    
    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                param.data = self.backup[name]
        self.backup = {}

def visualize_sr_enhancement(model, dataset, device, num_samples=8):
    """Visualize SR enhancement: LR → SR → HR comparison"""
    model.eval()
    samples = []
    for i in range(num_samples):
        sample = dataset[np.random.randint(len(dataset))]
        samples.append(sample)
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 3*num_samples))
    
    # Handle DataParallel wrapper
    sr_model = model.module.sr_model if hasattr(model, 'module') else model.sr_model
    
    with torch.no_grad():
        for idx, sample in enumerate(samples):
            lr_img = sample['lr'].unsqueeze(0).to(device)
            
            # Generate SR
            sr_img = sr_model(lr_img)
            
            # Denormalize for visualization
            lr_np = (lr_img[0].cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5).clip(0, 1)
            sr_np = (sr_img[0].cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5).clip(0, 1)
            
            # Plot LR
            axes[idx, 0].imshow(lr_np)
            axes[idx, 0].set_title(f'LR Input {lr_img.shape[2]}x{lr_img.shape[3]}')
            axes[idx, 0].axis('off')
            
            # Plot SR
            axes[idx, 1].imshow(sr_np)
            axes[idx, 1].set_title(f'SR Output {sr_img.shape[2]}x{sr_img.shape[3]}')
            axes[idx, 1].axis('off')
            
            # Plot difference (enhanced details)
            sr_resized = F.interpolate(sr_img, size=lr_img.shape[2:], mode='bilinear', align_corners=False)
            diff = torch.abs(sr_resized - lr_img)[0].cpu().numpy().mean(axis=0)
            axes[idx, 2].imshow(diff, cmap='hot')
            axes[idx, 2].set_title('Enhancement (Difference)')
            axes[idx, 2].axis('off')
    
    plt.tight_layout()
    wandb.log({"sr_enhancement_examples": wandb.Image(plt)})
    plt.savefig('/kaggle/working/sr_enhancement.png', dpi=300, bbox_inches='tight')
    plt.close()

def visualize_feature_maps(model, sample_input, device):
    """Visualize intermediate feature maps from ResNet"""
    model.eval()
    
    # Handle DataParallel wrapper
    if hasattr(model, 'module'):
        sr_model = model.module.sr_model
        backbone = model.module.backbone
    else:
        sr_model = model.sr_model
        backbone = model.backbone
    
    with torch.no_grad():
        # Get SR image
        sr_img = sr_model(sample_input)
        sr_img = F.interpolate(sr_img, size=(224, 224), mode='bilinear', align_corners=False)
        
        # Extract features at different layers
        x = backbone.conv1(sr_img)
        x = backbone.bn1(x)
        x = backbone.relu(x)
        x = backbone.maxpool(x)
        
        layer1_out = backbone.layer1(x)
        layer2_out = backbone.layer2(layer1_out)
        layer3_out = backbone.layer3(layer2_out)
        layer4_out = backbone.layer4(layer3_out)
        
        layers = [x, layer1_out, layer2_out, layer3_out, layer4_out]
        layer_names = ['Conv1+Pool', 'Layer1', 'Layer2', 'Layer3', 'Layer4']
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        # Plot input
        img_np = (sr_img[0].cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5).clip(0, 1)
        axes[0].imshow(img_np)
        axes[0].set_title('SR Input to ResNet', fontsize=14, fontweight='bold')
        axes[0].axis('off')
        
        # Plot feature maps
        for idx, (layer_out, name) in enumerate(zip(layers, layer_names)):
            # Average across channels
            feat_map = layer_out[0].cpu().numpy().mean(axis=0)
            im = axes[idx+1].imshow(feat_map, cmap='viridis')
            axes[idx+1].set_title(f'{name}\n{layer_out.shape[1]} channels, {layer_out.shape[2]}x{layer_out.shape[3]}', 
                                  fontsize=12, fontweight='bold')
            axes[idx+1].axis('off')
            plt.colorbar(im, ax=axes[idx+1], fraction=0.046)
        
        plt.tight_layout()
        wandb.log({"feature_maps": wandb.Image(plt)})
        plt.savefig('/kaggle/working/feature_maps.png', dpi=300, bbox_inches='tight')
        plt.close()

def plot_roc_curves(y_true, y_pred_probs, num_classes=19):
    """Plot ROC curves for multi-class classification"""
    from sklearn.preprocessing import label_binarize
    from sklearn.metrics import roc_curve, auc
    
    # Binarize labels
    y_true_bin = label_binarize(y_true, classes=list(range(num_classes)))
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot ROC for each class (show subset)
    ax1 = plt.subplot(2, 2, 1)
    for i in range(min(5, num_classes)):  # Show first 5 classes
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
        roc_auc = auc(fpr, tpr)
        ax1.plot(fpr, tpr, lw=2, label=f'Class {i} (AUC={roc_auc:.3f})')
    ax1.plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC Curves (First 5 Classes)')
    ax1.legend(loc='lower right', fontsize=8)
    ax1.grid(True, alpha=0.3)
    
    # Compute macro-average ROC
    ax2 = plt.subplot(2, 2, 2)
    all_fpr = np.unique(np.concatenate([roc_curve(y_true_bin[:, i], y_pred_probs[:, i])[0] 
                                        for i in range(num_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
        mean_tpr += np.interp(all_fpr, fpr, tpr)
    mean_tpr /= num_classes
    macro_auc = auc(all_fpr, mean_tpr)
    ax2.plot(all_fpr, mean_tpr, 'b-', lw=3, label=f'Macro-avg (AUC={macro_auc:.3f})')
    ax2.plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('Macro-Average ROC Curve')
    ax2.legend(loc='lower right')
    ax2.grid(True, alpha=0.3)
    
    # AUC distribution
    ax3 = plt.subplot(2, 2, 3)
    aucs = [auc(*roc_curve(y_true_bin[:, i], y_pred_probs[:, i])[:2]) for i in range(num_classes)]
    ax3.bar(range(num_classes), aucs, color='steelblue', alpha=0.7, edgecolor='navy')
    ax3.axhline(y=np.mean(aucs), color='red', linestyle='--', lw=2, label=f'Mean={np.mean(aucs):.3f}')
    ax3.set_xlabel('Class')
    ax3.set_ylabel('AUC')
    ax3.set_title('Per-Class AUC Distribution')
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    
    # AUC statistics
    ax4 = plt.subplot(2, 2, 4)
    ax4.axis('off')
    stats_text = f"""
    ROC-AUC Statistics:
    
    Macro-Average AUC: {macro_auc:.4f}
    Mean AUC: {np.mean(aucs):.4f}
    Std AUC: {np.std(aucs):.4f}
    Min AUC: {np.min(aucs):.4f} (Class {np.argmin(aucs)})
    Max AUC: {np.max(aucs):.4f} (Class {np.argmax(aucs)})
    
    Classes with AUC > 0.9: {sum(auc > 0.9 for auc in aucs)}/{num_classes}
    Classes with AUC > 0.8: {sum(auc > 0.8 for auc in aucs)}/{num_classes}
    """
    ax4.text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center', 
             fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    wandb.log({"roc_curves": wandb.Image(plt)})
    plt.savefig('/kaggle/working/roc_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return macro_auc, aucs

def plot_precision_recall_curves(y_true, y_pred_probs, num_classes=19):
    """Plot Precision-Recall curves"""
    from sklearn.preprocessing import label_binarize
    from sklearn.metrics import precision_recall_curve, average_precision_score
    
    y_true_bin = label_binarize(y_true, classes=list(range(num_classes)))
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Per-class PR curves (first 5)
    for i in range(min(5, num_classes)):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_pred_probs[:, i])
        ap = average_precision_score(y_true_bin[:, i], y_pred_probs[:, i])
        axes[0].plot(recall, precision, lw=2, label=f'Class {i} (AP={ap:.3f})')
    axes[0].set_xlabel('Recall')
    axes[0].set_ylabel('Precision')
    axes[0].set_title('Precision-Recall Curves (First 5 Classes)')
    axes[0].legend(loc='lower left', fontsize=9)
    axes[0].grid(True, alpha=0.3)
    
    # Average Precision distribution
    aps = [average_precision_score(y_true_bin[:, i], y_pred_probs[:, i]) for i in range(num_classes)]
    axes[1].bar(range(num_classes), aps, color='coral', alpha=0.7, edgecolor='darkred')
    axes[1].axhline(y=np.mean(aps), color='blue', linestyle='--', lw=2, label=f'Mean AP={np.mean(aps):.3f}')
    axes[1].set_xlabel('Class')
    axes[1].set_ylabel('Average Precision')
    axes[1].set_title('Per-Class Average Precision')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    wandb.log({"precision_recall_curves": wandb.Image(plt)})
    plt.savefig('/kaggle/working/pr_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return np.mean(aps), aps

def plot_learning_dynamics(history):
    """Plot comprehensive learning dynamics"""
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # Loss with smoothing
    ax1 = fig.add_subplot(gs[0, 0])
    epochs = range(1, len(history['train_loss']) + 1)
    ax1.plot(epochs, history['train_loss'], 'o-', alpha=0.6, label='Train', markersize=4)
    ax1.plot(epochs, history['val_loss'], 's-', alpha=0.6, label='Val', markersize=4)
    # Add moving average
    if len(history['train_loss']) > 5:
        train_smooth = np.convolve(history['train_loss'], np.ones(5)/5, mode='valid')
        val_smooth = np.convolve(history['val_loss'], np.ones(5)/5, mode='valid')
        ax1.plot(range(3, len(train_smooth)+3), train_smooth, 'b-', lw=2, alpha=0.8, label='Train (MA)')
        ax1.plot(range(3, len(val_smooth)+3), val_smooth, 'r-', lw=2, alpha=0.8, label='Val (MA)')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss Evolution with Moving Average')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy gap
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(epochs, history['train_acc'], 'g-', label='Train', lw=2)
    ax2.plot(epochs, history['val_acc'], 'b-', label='Val', lw=2)
    gap = np.array(history['train_acc']) - np.array(history['val_acc'])
    ax2.fill_between(epochs, history['val_acc'], history['train_acc'], alpha=0.3, color='orange', label='Gap')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.set_title(f'Train-Val Gap (Mean: {gap.mean():.2f}%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # F1 Score progression
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.plot(epochs, history['val_f1'], 'mo-', lw=2, markersize=6, label='Val F1')
    best_f1_epoch = np.argmax(history['val_f1']) + 1
    ax3.axvline(x=best_f1_epoch, color='red', linestyle='--', alpha=0.5, label=f'Best at epoch {best_f1_epoch}')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('F1 Score (%)')
    ax3.set_title(f'F1 Macro (Best: {max(history["val_f1"]):.2f}%)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Learning rate schedule
    ax4 = fig.add_subplot(gs[1, 0])
    ax4.plot(epochs, history['lr'], 'r-', lw=2)
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Learning Rate')
    ax4.set_title('Learning Rate Schedule')
    ax4.set_yscale('log')
    ax4.grid(True, alpha=0.3)
    
    # Loss landscape
    ax5 = fig.add_subplot(gs[1, 1])
    scatter = ax5.scatter(history['train_loss'], history['val_loss'], 
                         c=range(len(history['train_loss'])), cmap='viridis', 
                         s=100, alpha=0.6, edgecolors='black')
    ax5.plot([min(history['train_loss']), max(history['train_loss'])], 
            [min(history['train_loss']), max(history['train_loss'])], 
            'r--', alpha=0.5, label='y=x')
    ax5.set_xlabel('Train Loss')
    ax5.set_ylabel('Val Loss')
    ax5.set_title('Loss Correlation')
    plt.colorbar(scatter, ax=ax5, label='Epoch')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # Convergence rate
    ax6 = fig.add_subplot(gs[1, 2])
    if len(history['val_acc']) > 1:
        acc_improvement = np.diff(history['val_acc'])
        ax6.bar(range(1, len(acc_improvement)+1), acc_improvement, 
               color=['green' if x > 0 else 'red' for x in acc_improvement], alpha=0.7)
        ax6.axhline(y=0, color='black', linestyle='-', lw=1)
        ax6.set_xlabel('Epoch')
        ax6.set_ylabel('Accuracy Change (%)')
        ax6.set_title('Per-Epoch Accuracy Improvement')
        ax6.grid(True, alpha=0.3, axis='y')
    
    # Training statistics summary
    ax7 = fig.add_subplot(gs[2, :])
    ax7.axis('off')
    stats = f"""
    Training Statistics Summary:
    
    Best Validation Accuracy: {max(history['val_acc']):.2f}% (Epoch {np.argmax(history['val_acc'])+1})
    Best Validation F1: {max(history['val_f1']):.2f}% (Epoch {np.argmax(history['val_f1'])+1})
    Final Train Accuracy: {history['train_acc'][-1]:.2f}%
    Final Val Accuracy: {history['val_acc'][-1]:.2f}%
    Final Train-Val Gap: {history['train_acc'][-1] - history['val_acc'][-1]:.2f}%
    
    Loss Reduction: {history['train_loss'][0]:.4f} → {history['train_loss'][-1]:.4f} ({(1-history['train_loss'][-1]/history['train_loss'][0])*100:.1f}% decrease)
    Accuracy Gain: {history['val_acc'][0]:.2f}% → {history['val_acc'][-1]:.2f}% (+{history['val_acc'][-1]-history['val_acc'][0]:.2f}%)
    
    Learning Rate: {history['lr'][0]:.2e} → {history['lr'][-1]:.2e}
    Total Epochs Trained: {len(history['train_loss'])}
    """
    ax7.text(0.1, 0.5, stats, fontsize=11, verticalalignment='center', 
            fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
    
    plt.tight_layout()
    wandb.log({"learning_dynamics": wandb.Image(plt)})
    plt.savefig('/kaggle/working/learning_dynamics.png', dpi=300, bbox_inches='tight')
    plt.close()

print("✓ Comprehensive evaluation metrics, EMA & advanced visualizations defined")

In [ ]:
# ===============================================================================
# CELL 6: Full Training Pipeline with Active Learning
# ===============================================================================

# Setup with label smoothing and warmup
criterion = nn.CrossEntropyLoss(label_smoothing=config['label_smoothing'])
optimizer = optim.AdamW(classifier.parameters(), lr=config['lr'], 
                        weight_decay=config['weight_decay'])

# Warmup + Cosine Annealing scheduler
warmup_scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, 
                                               total_iters=config['warmup_epochs'])
main_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                      T_max=config['clf_epochs'] - config['warmup_epochs'],
                                                      eta_min=1e-6)
scheduler = optim.lr_scheduler.SequentialLR(optimizer, 
                                            schedulers=[warmup_scheduler, main_scheduler],
                                            milestones=[config['warmup_epochs']])

scaler = GradScaler() if config['mixed_precision'] else None

# Initialize EMA
ema = EMA(classifier, decay=config['ema_decay'])

# Training history
history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': [], 'val_f1': [],
    'lr': []
}

print("\n" + "="*80)
print("STARTING OPTIMIZED TRAINING PIPELINE")
print("="*80)
print(f"📊 Enhancements: Label Smoothing, Warmup LR, EMA, Enhanced Metrics")
print(f"⏱️  Estimated Time: ~2.5-3 hours for 30 epochs")
print("="*80)

# Track training time
start_time = time.time()

# Initial supervised training with enhanced evaluation
best_val_acc = 0
best_val_f1 = 0

for epoch in range(config['clf_epochs']):
    epoch_start = time.time()
    print(f"\nEpoch {epoch+1}/{config['clf_epochs']}")
    print("-" * 80)
    
    # Train
    train_loss, train_acc = train_epoch(classifier, train_loader, criterion, 
                                        optimizer, scaler, device)
    
    # Update EMA
    ema.update()
    
    # Validate with original weights
    val_loss, val_acc, val_f1, val_preds, val_labels = evaluate(classifier, val_loader, 
                                                                 criterion, device)
    
    # Validate with EMA weights
    ema.apply_shadow()
    val_loss_ema, val_acc_ema, val_f1_ema, val_preds_ema, val_labels_ema = evaluate(
        classifier, val_loader, criterion, device)
    ema.restore()
    
    scheduler.step()
    
    # Use EMA results if better
    use_ema = val_acc_ema > val_acc
    final_val_acc = val_acc_ema if use_ema else val_acc
    final_val_f1 = val_f1_ema if use_ema else val_f1
    final_val_loss = val_loss_ema if use_ema else val_loss
    final_preds = val_preds_ema if use_ema else val_preds
    
    # Store history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(final_val_loss)
    history['val_acc'].append(final_val_acc)
    history['val_f1'].append(final_val_f1)
    history['lr'].append(optimizer.param_groups[0]['lr'])
    
    # Compute per-class metrics
    precision, recall, f1_per_class, support = compute_per_class_metrics(
        val_labels_ema if use_ema else val_labels, final_preds, config['num_classes']
    )
    
    # Log metrics
    wandb.log({
        'epoch': epoch + 1,
        'train/loss': train_loss,
        'train/accuracy': train_acc,
        'val/loss': final_val_loss,
        'val/accuracy': final_val_acc,
        'val/f1_macro': final_val_f1,
        'val/precision_macro': precision.mean() * 100,
        'val/recall_macro': recall.mean() * 100,
        'val_ema/accuracy': val_acc_ema,
        'val_ema/f1_macro': val_f1_ema,
        'lr': optimizer.param_groups[0]['lr'],
        'epoch_time': time.time() - epoch_start
    })
    
    print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
    print(f"Val   - Loss: {final_val_loss:.4f}, Acc: {final_val_acc:.2f}%, F1: {final_val_f1:.2f}%")
    if use_ema:
        print(f"        (Using EMA weights - {val_acc_ema:.2f}% vs {val_acc:.2f}%)")
    print(f"        Precision: {precision.mean()*100:.2f}%, Recall: {recall.mean()*100:.2f}%")
    print(f"        LR: {optimizer.param_groups[0]['lr']:.2e}, Time: {time.time()-epoch_start:.1f}s")
    
    # Save best model (both standard and EMA)
    if final_val_acc > best_val_acc:
        best_val_acc = final_val_acc
        best_val_f1 = final_val_f1
        if use_ema:
            ema.apply_shadow()
            torch.save(classifier.state_dict(), '/kaggle/working/best_classifier.pth')
            ema.restore()
        else:
            torch.save(classifier.state_dict(), '/kaggle/working/best_classifier.pth')
        print(f"✓ Saved best model (acc: {best_val_acc:.2f}%, F1: {best_val_f1:.2f}%)")
    
    # Comprehensive visualizations based on viz_frequency
    viz_freq = config.get('viz_frequency', 5)
    is_viz_epoch = (epoch + 1) % viz_freq == 0 or (epoch + 1) in [1, 5, 10, 25, config['clf_epochs']]
    
    if is_viz_epoch:
        print(f"\n📊 Generating visualizations for epoch {epoch+1}...")
        
        # Standard visualizations
        plot_confusion_matrix(val_labels_ema if use_ema else val_labels, final_preds, epoch + 1)
        plot_per_class_performance(precision, recall, f1_per_class, support)
        plot_training_curves(history)
        
        # SR Enhancement examples
        if config.get('save_sr_examples', True):
            print("  → SR enhancement comparison...")
            visualize_sr_enhancement(classifier, train_dataset, device, num_samples=6)
        
        # Feature maps
        if config.get('save_feature_maps', True) and (epoch + 1) % (viz_freq * 2) == 0:
            print("  → Feature map extraction...")
            sample_batch = next(iter(val_loader))
            sample_input = sample_batch['lr'][:1].to(device)
            visualize_feature_maps(classifier, sample_input, device)
        
        # Learning dynamics (after epoch 2)
        if epoch > 1:
            print("  → Learning dynamics analysis...")
            plot_learning_dynamics(history)
        
        print("  ✓ Visualizations saved")
    
    # Memory cleanup
    if (epoch + 1) % 3 == 0:
        torch.cuda.empty_cache()
        gc.collect()

print("\n" + "="*80)
print(f"TRAINING COMPLETE")
print(f"Best Val Accuracy: {best_val_acc:.2f}%")
print(f"Best Val F1 Score: {best_val_f1:.2f}%")
print("="*80)

# Final comprehensive evaluation
print("\n" + "="*80)
print("FINAL EVALUATION WITH BEST MODEL")
print("="*80)

# Load best model
classifier.load_state_dict(torch.load('/kaggle/working/best_classifier.pth'))
val_loss, val_acc, val_f1, val_preds, val_labels = evaluate(classifier, val_loader, 
                                                             criterion, device)

# Compute all metrics
precision, recall, f1_per_class, support = compute_per_class_metrics(
    val_labels, val_preds, config['num_classes']
)

print(f"\n📊 Overall Metrics:")
print(f"  Accuracy:  {val_acc:.2f}%")
print(f"  F1 (Macro): {val_f1:.2f}%")
print(f"  Precision: {precision.mean()*100:.2f}%")
print(f"  Recall:    {recall.mean()*100:.2f}%")

print(f"\n📈 Per-Class Statistics:")
print(f"  Best F1 Class:   Class {f1_per_class.argmax()} ({f1_per_class.max()*100:.2f}%)") 
print(f"  Worst F1 Class:  Class {f1_per_class.argmin()} ({f1_per_class.min()*100:.2f}%)")
print(f"  F1 Std Dev:      {f1_per_class.std()*100:.2f}%")

# Generate final visualizations
print("\n📈 Generating comprehensive final visualizations...")

# 1. Confusion matrix
print("  → Confusion matrix...")
plot_confusion_matrix(val_labels, val_preds, 'FINAL')

# 2. Per-class performance
print("  → Per-class performance bars...")
plot_per_class_performance(precision, recall, f1_per_class, support)

# 3. Training curves
print("  → Training progression curves...")
plot_training_curves(history)

# 4. Learning dynamics
print("  → Learning dynamics analysis...")
plot_learning_dynamics(history)

# 5. Get prediction probabilities for ROC/PR curves
print("  → Computing prediction probabilities...")
classifier.eval()
all_probs = []
with torch.no_grad():
    for batch in tqdm(val_loader, desc='Computing probabilities'):
        lr_imgs = batch['lr'].to(device)
        outputs = classifier(lr_imgs)
        probs = torch.softmax(outputs, dim=1)
        all_probs.extend(probs.cpu().numpy())
all_probs = np.array(all_probs)

# 6. ROC curves
print("  → ROC curves...")
macro_auc, class_aucs = plot_roc_curves(val_labels, all_probs, num_classes=config['num_classes'])

# 7. Precision-Recall curves
print("  → Precision-Recall curves...")
mean_ap, class_aps = plot_precision_recall_curves(val_labels, all_probs, num_classes=config['num_classes'])

# 8. SR enhancement examples
print("  → Final SR enhancement examples...")
visualize_sr_enhancement(classifier, val_dataset, device, num_samples=8)

# 9. Feature maps
print("  → Final feature map visualization...")
sample_batch = next(iter(val_loader))
sample_input = sample_batch['lr'][:1].to(device)
visualize_feature_maps(classifier, sample_input, device)

# Classification report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(val_labels, val_preds, digits=4))

# Save classification report
report = classification_report(val_labels, val_preds, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv('/kaggle/working/classification_report.csv')
print("\n✓ Saved classification report to classification_report.csv")

# Log final comprehensive metrics
wandb.log({
    'final/accuracy': val_acc,
    'final/f1_macro': val_f1,
    'final/roc_auc_macro': macro_auc,
    'final/mean_average_precision': mean_ap,
    'final/training_hours': (time.time() - start_time) / 3600,
    'final/best_accuracy': best_val_acc,
    'final/best_f1': best_val_f1
})

print("\n" + "="*80)
print("COMPREHENSIVE EVALUATION COMPLETE")
print("="*80)
print(f"📊 Final Metrics Summary:")
print(f"  Validation Accuracy:     {val_acc:.2f}%")
print(f"  Validation F1 (Macro):   {val_f1:.2f}%")
print(f"  ROC-AUC (Macro):         {macro_auc:.4f}")
print(f"  Mean Average Precision:  {mean_ap:.4f}")
print(f"  Training Time:           {(time.time() - start_time)/3600:.2f} hours")
print(f"  Best Accuracy Achieved:  {best_val_acc:.2f}%")
print(f"  Best F1 Achieved:        {best_val_f1:.2f}%")
print("="*80)

wandb.finish()
print("\n✅ Training pipeline complete with comprehensive visual evaluation!")


STARTING FULL TRAINING PIPELINE

Epoch 1/20


Training:   0%|          | 0/724 [00:00<?, ?it/s]

KeyboardInterrupt: 